<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_04_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 4: ChatGPT and Large Language Models**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 4 Material

* **Part 4.1: Introduction to LLMs (ChatGTP) and Prompt Engineering**
* Part 4.2: Generative AI
* Part 4.3: Text to Speech
* Part 4.4: Text to Images


## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image01B.png)

If your GMAIL address does not appear your lesson will **not** be graded.

# **You MUST Purchase Your OpenAI API Key Now!**

In order to run the code in the next few lessons, you will need to purchase an `OpenAI API key` and install your key in the `secretes` location in your Google COLAB notebook. It is important to key your `OpenAI API key` secrete. If anyone learns your key, they can use it costing you a lot of money.  

## **How to Purchase an OpenAI API Key**

Follow these steps to obtain an API key from OpenAI:

**1. Create an OpenAI Account**

If you don’t already have one, go to https://platform.openai.com/api-keys and sign up using your email, Google, or Microsoft account.


**2. Log In to the OpenAI Platform**

Visit https://platform.openai.com/api-keys and log in with your credentials. After you log in you should see this page.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image17BB.png)

**3. Set Up Billing**

Before you can use the API you will need to add a payment method. Click on the **Settings** (gear) icon
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image18BB.png)

Then click on **Billing** on the left tab.
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image14BB.png)

The select **Payment methods** and enter your credit card information.
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image21BB.png)

When you are finished adding your credit card information you need to add some money to your account. Click on **Add to Credit Balance**
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image20BB.png)

Don't add too much money to your acccount since you can always add more money later if you use up your "tokens". Start with \$10-\$20.
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image25BB.png)

Also keep in mind that if your secret key gets stolen, all of your money might be quicky used up by the person who stole it!

**4. Generate an API Key**

When you have added all of your credit card information and added some money to your account, you are ready to generate your API key.

Select `API Keys` on the left tab
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image23BB.png)

Click on **Create new secret key** and follow the directions. The name you give your key is not important. I just called my key `FALL25`.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image24BB.png)

Your new API key will look something like this:

`sk-proj-DHIIXTfLozih_0rjgx08wxExjMc0G8SDqSyVtUG5pur0pVMLF5XgO2-TX2l9ZnXOiHk7Wv4sdvT4BlbkFJGxLnypUPLYpyYzVI6aOiZMx_1LOMKdADziRqlydDTg3HpwW0g7bSNmzuy0Tb0uzeyeCbz9qNMB`

**WARNING:**
You need to _immediately_ copy-and-paste your API into either the **Notepad** app if you are working on a Windows PC or the **TextEdit** app if you have a MAC.  

> ⚠️ **Important:** Never share your API key publicly or commit it to version control.

**5. Add Your API Key to Colab Secrets**

In the Colab menu:

Click on the key symbol on the left tab. This is your `secrets` location

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image02B.png)


Click on `+ Add new secret`

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image04B.png)


In the open box _carefully_ type the word `OPENAI_API_KEY`.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image06B.png)

**WARNING:** Make sure that you spell and capitalize the word **_exactly_** as `OPENAI_API_KEY` or you secret key will not work!

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image08B.png)


Paste your actual API key into the Value field.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image10B.png)

Don't add any quotation marks to your API key. Just paste the actual key.

When you have pasted your key in the `Value` box, click on the `+ Add new secret`


![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image12B.png)

Make sure to click on `Notebook access` so you use your secret `OPENAI_API_KEY` in this lesson.

Finally, click on the **X** at the top right of the **Secrets** panel to restore your COLAB notebook.




## **Test Your `OPENAI_API_KEY`**

To see if your `OPENAI_API_KEY` is correctly setup, run the next code cell.

In [ ]:
from google.colab import userdata
#from langchain_openai import OpenAI, ChatOpenAI

from google.colab import userdata
OPENAI_KEY = userdata.get('OPENAI_API_KEY')

# Retrieve the OpenAI API key and store it in a variable
#OPENAI_KEY = userdata.get('OPENAI_KEY')

# Ensure that the API key is correctly set
if not OPENAI_KEY:
    raise ValueError("OpenAI API key is not set. Please check if you have stored the API key in userdata.")
else:
  print(f"Your secret OPENAI_API_KEY =", OPENAI_KEY)

If your `OPENAI_API_KEY` is correctly installed you should see the following output.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image26BB.png)

However, if you see the following output
```type
OpenAI API key is not set. Please check if you have stored the API key in userdata.
```
You will need to correct the error before you can continue. Ask your Instructor or TA for help if you can resolve the error yourself.

# **Introduction to Large Language Models (LLMs)**

**Large Language Models (LLMs)** such as `GPT` have brought AI into mainstream use. LLMs allow regular users to interact with AI using natural language. Most of these language models require extreme processing capabilities and hardware. Because of this, application programming interfaces (APIs) accessed through the Internet are becoming common entry points for these models. One of the most compelling features of services like ChatGPT is their availability as an API. But before we dive into the depths of coding and integration, let's understand what an API is and its significance in the AI domain.

API stands for **Application Programming Interface**. Think of it as a bridge or a messenger that allows two different software applications to communicate. In the context of AI and machine learning, APIs often allow developers to access a particular model or service without having to house the model on their local machine. This technique can be beneficial when the model in question, like `ChatGPT`, is large and resource-intensive.

In the realm of AI, APIs have several distinct advantages:

**1 Scalability:** Since the actual model runs on external servers, developers don't need to worry about scaling infrastructure.  
**2. Maintenance:** You get to use the latest and greatest version of the model without constantly updating your local copy.  
**3. Cost-Effective:** Leveraging external computational resources can be more cost-effective than maintaining high-end infrastructure locally, especially for sporadic or one-off tasks.  
**4 Ease of Use:** Instead of diving into the nitty-gritty details of model implementation and optimization, developers can directly utilize its capabilities with a few lines of code.  

In this section, we won't be running the neural network computations locally. We will use our PyTorch code to communicate with the `OpenAI API` to access and harness the abilities of `ChatGPT`. The actual execution of the neural network code happens on `OpenAI servers`, bringing forth a unique synergy of PyTorch's flexibility and ChatGPT's conversational mastery. (NOTE: The physical location of these servers is not disclosed for security reasons).

In this section, we will make use of the `OpenAI ChatGPT API`. Further information on this API can be found here:

* [OpenAI API Login/Registration](https://platform.openai.com/apps)
* [OpenAI API Reference](https://platform.openai.com/docs/introduction/overview)
* [OpenAI Python API Reference](https://platform.openai.com/docs/api-reference/introduction?lang=python)
* [OpenAI Python Library](https://github.com/openai/openai-python)
* [OpenAI Cookbook for Python](https://github.com/openai/openai-cookbook/)
* [LangChain](https://www.langchain.com/)


## **Installing LangChain to use the OpenAI Python Library**

As we delve deeper into the intricacies of deep learning, it's crucial to understand that the tools and platforms we use are as versatile as the concepts themselves. When it comes to accessing ChatGPT, a state-of-the-art conversational AI model developed by OpenAI, there are two predominant pathways:

**Direct API Access using Python's HTTP Capabilities:** Python, with its rich library ecosystem, provides utilities like requests to directly communicate with APIs over HTTP. This method involves crafting the necessary API calls, handling responses, and error checking, giving the developer a granular control over the process.

**Using the Official OpenAI Python Library:** OpenAI offers an official Python library, aptly named openai, that simplifies the process of integrating with ChatGPT and other OpenAI services. This library abstracts many of the intricacies and boilerplate steps of direct API access, offering a streamlined and user-friendly approach to interacting with the model.

Each approach has its advantages. `Direct API access` provides a more hands-on, granular approach, allowing developers to intimately understand the intricacies of each API call. On the other hand, using the `openai library` can accelerate development, reduce potential errors, and allow for a more straightforward integration, especially for those new to API interactions.

We will make use of the `OpenAI API` through a library called `LangChain`. `LangChain` is a framework designed to simplify the creation of applications using LLMs. As a language model integration framework, `LangChain's` use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis. `LangChain` allows you to quickly change between different underlying LLMs with minimal code changes.


### Install `LangChain`

The following command installs the **LangChain** library and needed OpenAI LLM connectors.

In [ ]:
# Install packages

!pip install langchain langchain_openai > /dev/null
!pip install langchain-community > /dev/null

If the code is correct you should not see any output.

## **OpenAI Models that are Accessible via API**

`OpenAI` currently provides access to several models via its API, catering to a variety of tasks. Here's an overview:

**1. GPT Models**
- **GPT-4.5**: The largest and most advanced model, designed for creative tasks and agentic planning.
- **GPT-4o**: A high-intelligence model for complex tasks, supporting text and vision inputs with a 128k context length.
- **GPT-4o Mini**: A smaller, cost-efficient model optimized for lightweight tasks, also supporting text and vision inputs.

**2. Reasoning Models**
- **o1**: A frontier reasoning model supporting tools, structured outputs, and vision tasks with a 200k context length.
- **o3-mini**: A cost-efficient reasoning model optimized for coding, math, and science, with support for tools and structured outputs.

**3. API Endpoints**
- **Responses API**: A unified endpoint combining functionalities like text/image input, web/file search, and reasoning.
- **Chat Completions API**: For conversational AI tasks.
- **Realtime API**: Enables low-latency, multimodal experiences, including speech-to-speech.
- **Assistants API**: Builds AI assistants capable of complex, multi-step tasks.
- **Batch API**: Processes asynchronous workloads at reduced costs.

These models and tools are designed to address a wide range of use cases, from general-purpose tasks to specialized reasoning and coding challenges.

### **GPT-3.5-turbo-1106**

For this course, we will generally use the model `GTP-3.5-turbo-1106`.

The **GPT-3.5-turbo-1106** model is one of `OpenAI's` advanced generative AI models, recently launched as part of the Azure OpenAI Service. Here are some key details:

* **Availability:**
It was announced alongside GPT-4 Turbo at Microsoft Ignite 2023 and is now available globally through Azure OpenAI Service.

* **Performance:**
The model is designed to provide improved cost efficiency and generative capabilities for businesses. It supports a wide range of applications, including natural language understanding, text generation, and more.

* **Use Cases:**
It is optimized for tasks like conversational AI, content creation, and other applications requiring high-quality text generation.

### Set LLM for this Lesson

Run the next cell to specify `gpt-3.5-turbo-1106` as the specific LLM to use for this lesson.

In [ ]:
# This is the model you will generally use for this class

LLM_MODEL = 'gpt-3.5-turbo-1106'

print(f"The Large Language Model (LLM) is set for", LLM_MODEL)

If the code is correct your should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image27B.png)

## **Prompt Engineering**

When working with a large language model (LLM) like ChatGPT, the **prompt** serves as the foundation for interaction. It is the input or instruction provided to the model, guiding it to generate relevant and useful outputs.

**1. Role of the Prompt**
- **Instructional Guide**: The prompt shapes what the model does. Whether it's answering a question, completing a task, or writing creatively, the prompt provides the necessary context.  
- **Boundary Setter**: A well-crafted prompt can define the scope of the task, ensuring the response is focused and doesn't deviate from the topic.  
- **Task Optimizer**: By providing clear and concise instructions, the prompt ensures that the LLM generates responses that align with user expectations.

**2. Importance of the Prompt**
- **Determines Quality of Output**: The quality of the model's response depends heavily on the clarity and specificity of the prompt. A vague prompt can lead to irrelevant or incomplete answers, while a precise one produces accurate and valuable results.
- **Customizable Interactions**: Prompts allow users to adapt the model to different scenarios—such as summarization, translation, or brainstorming—making it versatile and dynamic.  
- **Reduces Ambiguity**: A good prompt minimizes room for misunderstanding, helping the model interpret the task as intended.  

**3. Iterative Improvement**
Working with LLMs is often an _iterative_  process. If the initial response isn't quite right, the user can refine the prompt, adding more detail or constraints to guide the model toward the desired result. Instead of starting over from scratch, you just edit the prompt and try it again.

The prompt isn't just the input—it’s the bridge between the user’s needs and the model’s capabilities. Mastering prompt design is key to fully leveraging the potential of an LLM like ChatGPT.


### Example 1: Basic Query to LangChain

We begin by writint a **prompt**, to ask (query) `ChatGPT` a simple question: "What are the 5 largest cities in the USA?".

The Python code in the cell below interacts with OpenAI's GPT model using `LangChain` and the `ChatOpenAI class` to retrieve our answer.

**NOTE:** This cell will not run if you do not have a valid OpenAI_Key and you have already installed your key with Google Colab.


In [ ]:
# Example 1: Basic Query

from google.colab import userdata
from langchain_openai import OpenAI, ChatOpenAI

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model=LLM_MODEL, temperature=0)

# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the OpenAI API
response = llm.invoke(question)

# Print the response
print(response.content)


If the code is correct your should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image28B.png)

As you can see, the response from `LangChain` is in regular English, complete with formatting. While the formatting may make it easier to read, we often have to parse the results given to us by LLMs.

Later, we will see that `LangChain` can help with this as well. You will also notice that we specified a value of `0` for **temperature**; this instructs the LLM to be less creative with its responses and more consistent. Because we are working primarily with data extraction in this section, a low temperature will give us more consistent results.

In `LangChain`, the temperature parameter typically ranges from **0.0** to **1.0**, though some implementations may allow values slightly above 1.0. The temperature controls the randomness of the model's output:

* **Low Temperature (e.g., 0.0):** Produces more deterministic and focused responses, ideal for tasks requiring precision.

* **High Temperature (e.g., 1.0):** Generates more creative and diverse outputs, useful for brainstorming or creative writing.

If you're working with `LangChain` and `OpenAI models`, you can set the temperature when initializing the model or during runtime.

### **Exercise 1: Basic Query to LangChain**

For **Exercise 1** think about a subject for a `Top Five List` that **you** find interesting and see what response you get back from `ChatGTP`.

Feel free to change the **temperature** of your request if you want a more _creative_ response from `LangChain`. There are no "right" or "wrong" answers here as long as your code works.

In [ ]:
# Insert your code for Exercise 1 here


Since I am interested in guitar players, I asked `LangChain's` for a list of the 5 greatest guitart players of all time.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image29B.png)

You output will be different depending up your question.

### Example 2: Working with Prompts

As mentioned above, interactions with LLMs is typically accomplished using `prompts`. In fact, there is a whole new field called **Prompt Engineering** that focuses on designing, refining, and optimizing prompts to maximize the effectiveness and relevance of outputs generated by large language models (LLMs) like ChatGPT, GPT-4, and others.

In Example 2, we will "engineer" a prompt that will have `ChatGPT` translate text from French to English. In this example, we will just be using normal Python F-Strings to build the prompt.


In [ ]:
# Example 2: Working with prompts

from langchain_openai import OpenAI, ChatOpenAI

# Define text and style
text = """Laissez les bons temps rouler"""  # French text
style = "American English"                  # English

# Build prompt
prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
# Send promt to ChatGPT
response = llm.invoke(prompt)

# Print ChatGPTs response
print(response.content)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image30B.png)

--------------------------

**Why does the code Uses Triple Quotes?**

The code in the cell above uses triple double quotes (""") for the prompt string to allow for clean, multi-line formatting and to include special characters, such as backticks (```) and placeholders ({style} and {text}).

~~~text
prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```"""
~~~

-------------------------


### **Exercise 2: Working with Prompts**

In the cell below, use ChatGPT to translate the German expression: "Ein Prosit der Gemütlichkeit" into English.


In [ ]:
# Insert your code for Exercise 2 here


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image31B.png)

## **Dynamic Prompts**

A **dynamic prompt** is a flexible and adaptive input designed for interaction with language models (LLMs) like `ChatGP`T, where placeholders or variables are used to customize the prompt based on context or user-provided information. This approach allows for reusability, personalization, and automation, ensuring that the output is tailored to specific needs without rewriting the entire prompt.

---

#### **Key Characteristics of a Dynamic Prompt**
1. **Variables**:
   - Dynamic prompts include placeholders for variables, like `{name}`, `{style}`, or `{text}`, which can be filled with different values at runtime.
   - For example:
     ```python
     prompt = f"Translate this text: {text} into {language}."
     ```
     Here, `{text}` and `{language}` can be dynamically replaced by the desired input values.
2. **Context-Aware**:
   - They adapt to the context, such as the user’s preferences, conversation history, or specific tasks.
   - For instance, a dynamic prompt for summarization might consider the length of the desired output: "Summarize the following article in less than {words} words."
3. **Reusable Templates**:
   - Instead of hardcoding individual tasks, dynamic prompts use templates that can be applied across multiple scenarios by simply replacing values.
   - Example template:
     ```python
     template_text = """Write a {tone} response to the following message:
     message: {user_message}"""
     ```
4. **Personalization**:
   - Dynamic prompts can be personalized based on user inputs or profiles, enhancing user experience. For example:
     ```python
     f"Hi {name}, here’s the weather forecast for {city}!"
     ```

#### **Why Are Dynamic Prompts Important?**

- **Efficiency**: They save time by enabling template reuse.
- **Scalability**: Useful for applications needing to handle diverse inputs.
- **Adaptability**: They produce tailored outputs depending on the specific context or task.
- **User Experience**: Personalization through dynamic prompts improves user satisfaction.

---

Dynamic prompts are at the heart of effective interactions with LLMs, making them more versatile, context-aware, and user-specific.

### Example 3 - Step 1: Build a Dynamic Prompt

We can use LangChain to help us build dynamic prompts.

The first step is provide LangChain with a `template prompt`. The code in the cell below defines and creates a prompt template using LangChain's `ChatPromptTemplate` class. The prompt template is called `example_prompt_template`.

In [ ]:
# Example 3 - Step 1: Create prompt template

from langchain.prompts import ChatPromptTemplate

# Define prompt template
template_text = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# Create template
example_prompt_template = ChatPromptTemplate.from_template(template_text)


If the code is correct, you shouldn't see any output.

### Example 3 - Step 2: Build Dynamic Prompt

Now we can fill in the blanks for this prompt and observe the prompt created, which is a text string.

The code in the cell below does the following:

* Dynamically generates a structured prompt based on a template.
* Ensures the prompt includes placeholders (style and text) filled with the
provided values.
* Inspects the data structure and type of the resulting prompt.
* Outputs the first message to verify its content.

This code is useful for building prompts in LangChain when interacting with language models for tasks like translation, summarization, or custom instructions

In [ ]:
# Example 3 - Step 2: Use template to create prompt

example_prompt = example_prompt_template.format_messages(
                    style="American English",
                    text="千里之行，始于足下。")

# Print prompt and its features
print(type(example_prompt))
print(type(example_prompt[0]))

print(example_prompt[0])


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image32B.png)

In this example, we are asking `ChatGPT` to translate the `text` "千里之行，始于足下。" into the `style` "American English".

### Example 3 - Step 3: Build Dynamic Prompt

Now that we have buit our dynamic prompt in Steps 1 and 2, we are ready to send it to `ChatGPT` for analysis as shown in the code below.

In [ ]:
# Example 3 - Step 3: Send prompt to llm for analysis

# Call the LLM to translate to the style of the customer message
example_response = llm.invoke(example_prompt)

# Print response from ChatGPT
print(example_response)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image33B.png)

This newly constructed prompt can now perform the intended task of translation.

### **Exercise 3 - Step 1: Build Dynamic Prompt**

In the cell below, create a prompt template called `exercise_prompt_template`.


In [ ]:
# Insert your code for Exercise 3 - Step 1 here


If the code is correct, you shouldn't see any output.

### **Exercise 3 - Step 2: Build Dynamic Prompt**

Suppose you are standing watch at the White House and you receive this urgent message: "Президент Трамп, русская Родина сдаётся". Use your `exercise_prompt_template` to translate this message into English.

In [ ]:
# Insert your code for Exercise 3 - Step 2 here


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image34B.png)

### **Exercise 3 - Step 3: Build Dynamic Prompt**

Finally, send your `exercise_prompt_template` to `ChatGPT` to translate the urgent message in English.

In [ ]:
# Insert your code for Exercise 3 - Step 3 here


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image35B.png)

## **LLM Memory**

Human minds have both long-term and short-term memory. Long-term memory is what the human has learned throughout their lifetime. Short-term memory is what a human has only recently discovered in the last minute or so. For humans, learning is converting short-term memory into long-term memory that we will retain.

This process works somewhat differently for a LLM. Long-term memory was the weight of the neural network when it was initially trained or finetuned. Short-term memory is additional information that we wish the LLM to retain from previous prompts. For example, if the first prompt is "My name is David", the LLM will likely tell you hello and repeat your name. However, the LLM will not know the answer if the second prompt is "What is my name." without adding a memory component.

These memory objects, which `LangChain` provides, provide a sort of short-term memory. It is important to note that these objects are not affecting the long-term memory of the LLM, and once you discard the memory object, the LLM will forget. Additionally, the memory object can only hold so much information; newer information may replace older information once it is filled.

One important point to remember is that LLM's only have their input prompt. To provide such memory, these objects are appending anything we wish the LLM to remember to the input prompt. This section will see two ways to augment the prompt with previous information: a buffer and a summary. The buffer prepends a script of the last conversation up to this point. The summary approach keeps a consistently updated summary paragraph of the conversation.

### **Conversation Buffer Window Memory**

The `LangChain library` includes a conversation object named **ConversationChain**; this object facilitates an ongoing conversation with an LLM. For any conversation object, you must also specify a memory. For this first example, we will use the **ConversationBufferWindowMemory** object. This object keeps a transcript of the most recent conversation to reference. This memory allows the conversation object to remember what you have asked or told it and its responses to you.

In [ ]:
# Create buffer memory

from langchain.memory import ConversationBufferMemory
from langchain_community.retrievers import ZepCloudRetriever
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

# Create memory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history") # Important to set memory_key

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

# Create a chain
runnable = prompt | llm

def get_session_history(session_id: str): # Needed for RunnableWithMessageHistory
    return memory.load_memory_variables({})["chat_history"]  # Load full history


# Use the RunnableWithMessageHistory
conversation = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    history_buffer=memory,
    input_key="input",
    output_key="content"
)

memory = ConversationBufferWindowMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image53B.png)

### Talking to the LLM

We can now have a conversation with the LLM. This newly constructed prompt can now perform the intended task of translation.


In [ ]:
# Start Conversation

conversation.predict(input="Hi, my name is David")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image36B.png)

In [ ]:
conversation.predict(input="What is my name?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image37B.png)

We can have a look at what the memory now contains.


In [ ]:
conversation.memory.load_memory_variables({})

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image38B.png)

## **Custom Conversation Bots**

A **Custom Conversation Bot** is an AI-powered system that:
- Engages in natural language conversations
- Is customized for a specific domain, task, or personality
- Can be deployed on websites, apps, or messaging platforms

#### Key Features

- **Custom Instructions**: Define how the bot should behave, respond, and what tone it should use.
- **Knowledge Integration**: Connects to external data sources or APIs.
- **User Memory**: Can remember user preferences or past interactions.
- **Multimodal Capabilities**: Some bots can handle text, images, and even voice.

#### Common Use Cases

- Customer support
- Educational tutoring
- Personal assistants
- Sales and marketing
- Healthcare triage
- Entertainment and storytelling

#### Summary

Custom conversation bots are flexible, intelligent tools that can be tailored to meet specific needs, making them valuable across industries and applications.

### Example 4: Custom Conversational Bot

In Example 4 we are going to create a custom conversational bot named "UTSA Assistant" that is designed to help students at UTSA.

The first step is to create a `template` that defines the focus of the `Bot` and how it responds to questions.

```text
template = """The following is a friendly conversation between a human and an
AI to assist UTSA Students.
The AI should stick to topics about The University of Texas at San Antonion (UTSA).
If the AI does not know the answer to a question,
it should suggest the student speak to their advisor.
```
In this example, the focus of the Bot is the `University of Texas at San Antonio`.

To keep the memory of this `Bot` separate from other `Bots` that you might create, the following line of code is used to specify the `ai_prefix`:

```text
memory=ConversationBufferWindowMemory(ai_prefix="UTSA Assistant"),
```

In [ ]:
# Example 4: Create Custom Conversation Bot

from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an
AI to assist UTSA Students.
The AI should stick to topics about The University of Texas at San Antonion (UTSA).
If the AI does not know the answer to a question,
it should suggest the student speak to their advisor.

Current conversation:
{history}
Human: {input}
UTSA Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=False,
    memory=ConversationBufferWindowMemory(ai_prefix="UTSA Assistant"),
)

### Example 4: Conversing with Custom Bot

We can now have a conversation with our UTSA assistant bot.

### Example 4: Question 1

In [ ]:
# Example 4: Question 1

conversation.predict(input="Where is the bookstore?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image39B.png)

### Example 4: Question 2

In [ ]:
# Example 4: Question 2

conversation.predict(input="What is a nice quiet area to study?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image40B.png)

### Example 4: Question 3

In [ ]:
# Example 4: Question 3

conversation.predict(input="Which of these is closest to the bookstore?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image41B.png)

Our LLM_MODEL `gpt-3.5-turbo-1106` apparently knows a lot about UTSA! Let's see if our LLM knows about the most recent changes to our University like the appointment of the new Dean of the College of Sciences, Dr. Stephanie Santorico?

### Example 4: Question 4

In [ ]:
# Example 4: Question 4

conversation.predict(input="Who is the new Dean of the College of Sciences?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image42B.png)

You should keep in that LLMs are trained at a certain point in time. While there are some LLMs that can "scarp the web" for the latest data, LLMs models like `gpt-3.5-turbo-1106` have static memories. (As an aside, the University's Microsoft 365 Copilot did correctly identify our new Dean).

### Example 4: Question 5

Let's ask our our LLM_MODEL = 'gpt-3.5-turbo-1106' a more philosophical question.

In [ ]:
# Example 4: Question 5

conversation.predict(input="What is the meaning of life.")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image43B.png)

Our bot has a narrow focus. ☺

### Example 4: Print Out Memory

We can have a look at what the memory now contains.

In [ ]:
# Example 4: Print out memory

conversation.memory.load_memory_variables({})

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image44B.png)

## **Exercise 4: Custom Conversational Bot**

You are design your own **Custom Conversation Bot** and then ask it 5 questions that are within its field of focus. It is expected that every student will pick a **_different_** focus and use **_different_** questions (i.e. don't `copy-and-paste` from your coding friend).

### **Exercise 4: Create Chat Bot**

In the cell below, write the code to create your `Custom Chat Bot`.

In [ ]:
# Insert your code for Exercise 4: Create Custom Conversation Bot


### **Exercise 4: Questions**

Use the next 5 blank code cells to ask your Custom Chat Bot questions.  

### **Exercise 4: Question 1**

In [ ]:
# Insert your code for Exercise 4: Question 1 here



### **Exercise 4: Question 2**

In [ ]:
# Insert your code for Exercise 4: Question 2 here


### **Exercise 4: Question 3**

In [ ]:
# Insert your code for Exercise 4: Question 3 here


### **Exercise 4: Question 4**

In [ ]:
# Insert your code for Exercise 4: Question 4 here


### **Exercise 4: Question 5**

In [ ]:
# Insert your code for Exercise 4: Question 5 here


### **Exercise 4: Print Out Memory**

In [ ]:
# Insert your code for Exercise 4: Print History here


## **Conversation Summary Memory**

Now, let's look at using a slightly more complex type of memory, the `ConversationSummaryMemory` object. This type of memory creates a summary of the conversation over time. This memory can be helpful for condensing information from the conversation over time. Conversation summary memory summarizes the conversation and stores the current summary in memory. You can use this memory to inject the conversation summary so far into a prompt/chain. This memory is most useful for more extended conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

### Install Conversation Summary Memory

Run the code in the next cell to install `Conversation Summary Memory`.

In [ ]:
# Install conversation summary memory

from langchain.memory import ConversationSummaryMemory
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationChain

# Create a chat message history instance
chat_history = ChatMessageHistory()

# Initialize memory using from_messages
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=chat_history
)

# Create the conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)


### Simple Prompt Example

To illustrate the `ConversationSummaryMemory` function we will start with a simple prompt.

In [ ]:
# Simple prompt example

conversation.predict(input="I am a computational biologist, what do you do for a living?")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image45B.png)

### Examine Memory

The code in the next cell prints out the memory.

In [ ]:
# Examine memory

conversation.memory.load_memory_variables({})

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image46B.png)

## **What are Embedding Layers in PyTorch**

[Embedding Layers](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) are a handy feature of PyTorch that allows the program to automatically insert additional information into the data flow of your neural network. An embedding layer would automatically allow you to insert vectors in the place of word indexes.  

Programmers often use embedding layers with Natural Language Processing (NLP); however, you can use these layers when you wish to insert a lengthier vector in an index value place. In some ways, you can think of an embedding layer as dimension expansion. However, the hope is that these additional dimensions provide more information to the model and provide a better score.

## **Simple Embedding Layer Example**

* **num_embeddings** = How large is the vocabulary?  How many categories are you encoding? This parameter is the number of items in your "lookup table."
* **embedding_dim** = How many numbers in the vector you wish to return.

Now we create a neural network with a vocabulary size of 10, which will reduce those values between 0-9 to 4 number vectors. This neural network does nothing more than passing the embedding on to the output. But it does let us see what the embedding is doing. Each feature vector coming in will have two such features.

In [ ]:
# Simple embedding example

import torch
import torch.nn as nn

embedding_layer = nn.Embedding(num_embeddings=10, embedding_dim=4)
optimizer = torch.optim.Adam(embedding_layer.parameters(), lr=0.001)
loss_function = nn.MSELoss()

Let's take a look at the structure of this neural network to see what is happening inside it.

In [ ]:
# Print structure

print(embedding_layer)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image47B.png)

For this neural network, which is just an embedding layer, the input is a vector of size 2. These two inputs are integer numbers from 0 to 9 (corresponding to the requested input_dim quantity of 10 values). Looking at the summary above, we see that the embedding layer has 40 parameters. This value comes from the embedded lookup table that contains four amounts (output_dim) for each of the 10 (input_dim) possible integer values for the two inputs. The output is 2 (input_length) length 4 (output_dim) vectors, resulting in a total output size of 8, which corresponds to the Output Shape given in the summary above.

Now, let us query the neural network with two rows. The input is two integer values, as was specified when we created the neural network.

In [ ]:
# Query network

input_tensor = torch.tensor([[1, 2]], dtype=torch.long)
pred = embedding_layer(input_tensor)

print(input_tensor.shape)
print(pred)


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image48B.png)

Here we see two length-4 vectors that PyTorch looked up for each input integer. Recall that Python arrays are zero-based. PyTorch replaced the value of 1 with the second row of the 10 x 4 lookup matrix. Similarly, PyTorch returned the value of 2 by the third row of the lookup matrix. The following code displays the lookup matrix in its entirety. The embedding layer performs no mathematical operations other than inserting the correct row from the lookup table.


In [ ]:
# Print out embedding weights

embedding_layer.weight.data

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image49B.png)

The values above are random parameters that PyTorch generated as starting points. Generally, we will transfer an embedding or train these random values into something useful. The following section demonstrates how to embed a hand-coded embedding.

## **Transferring An Embedding**

Now, we see how to hard-code an embedding lookup that performs a simple one-hot encoding.  One-hot encoding would transform the input integer values of 0, 1, and 2 to the vectors $[1,0,0]$, $[0,1,0]$, and $[0,0,1]$ respectively. The following code replaced the random lookup values in the embedding layer with this one-hot coding-inspired lookup table.

In [ ]:
# Transfer embedding

import torch
import torch.nn as nn

# Define the embedding lookup matrix
embedding_lookup = torch.tensor([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
], dtype=torch.float32)  # Make sure to use float32 for weight matrices

# Create the embedding layer
embedding_layer = nn.Embedding(num_embeddings=3, embedding_dim=3)

# Set the weights of the embedding layer
embedding_layer.weight.data = embedding_lookup


We have the following parameters for the Embedding layer:
    
* `input_dim=3` - There are three different integer categorical values allowed.
* `output_dim=3` - Three columns represent a categorical value with three possible values per one-hot encoding.
* `input_length=2` - The input vector has two of these categorical values.

We query the neural network with two categorical values to see the lookup performed.

In [ ]:
# Create the input tensor directly in PyTorch

input_tensor = torch.tensor([[0, 1]], dtype=torch.long)

# Forward pass to get the predictions
pred = embedding_layer(input_tensor)

print(input_tensor.shape)
print(pred)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image50B.png)

The given output shows that we provided the program with two rows from the one-hot encoding table. This encoding is a correct one-hot encoding for the values 0 and 1, where there are up to 3 unique values possible.

The following section demonstrates how to train this embedding lookup table.

## **Training an Embedding**

First, we make use of the following imports.

In [ ]:
# Train embedding

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
from torch.nn.utils.rnn import pad_sequence

We create a neural network that classifies restaurant reviews according to positive or negative. This neural network can accept strings as input, such as given here. This code also includes positive or negative labels for each review.

In [ ]:
# Define 10 resturant reviews.

reviews = [
    'Never coming back!',
    'Horrible service',
    'Rude waitress',
    'Cold food.',
    'Horrible food!',
    'Awesome',
    'Awesome service!',
    'Rocks!',
    'poor work',
    'Couldn\'t have done better']

# Define labels (1=negative, 0=positive)
labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

Notice that the second to the last label is incorrect.  Errors such as this are not too out of the ordinary, as most training data could have some noise.

We define a vocabulary size of 50 words.  Though we do not have 50 words, it is okay to use a value larger than needed.  If there are more than 50 words, the least frequently used words in the training set are automatically dropped by the embedding layer during training.  For input, we one-hot encode the strings.  We use the TensorFlow one-hot encoding method here rather than Scikit-Learn. Scikit-learn would expand these strings to the 0's and 1's as we would typically see for dummy variables.  TensorFlow translates all words to index values and replaces each word with that index.

In [ ]:
# One-hot encode reviews

VOCAB_SIZE = 50
encoded_reviews = [torch.tensor([hash(word) % VOCAB_SIZE for word in review.split()]) for review in reviews]

print(f"Encoded reviews: {encoded_reviews}")

The program one-hot encodes these reviews to word indexes; however, their lengths are different. We pad these reviews to 4 words and truncate any words beyond the fourth word.


In [ ]:
# Pad length

MAX_LENGTH = 4
padded_reviews = pad_sequence(encoded_reviews, batch_first=True, padding_value=0).narrow(1, 0, MAX_LENGTH)
print(padded_reviews)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image51B.png)

As specified by the padding=post setting, each review is padded by appending zeros at the end, as specified by the padding=post setting.

Next, we create a neural network to learn to classify these reviews.


In [ ]:
# Create neural network

model = nn.Sequential(
    nn.Embedding(VOCAB_SIZE, 8),
    nn.Flatten(),
    nn.Linear(8 * MAX_LENGTH, 1),
    nn.Sigmoid()
)

This network accepts four integer inputs that specify the indexes of a padded movie review. The first embedding layer converts these four indexes into four length vectors 8. These vectors come from the lookup table that contains 50 (VOCAB_SIZE) rows of vectors of length 8. This encoding is evident by the 400 (8 times 50) parameters in the embedding layer. The output size from the embedding layer is 32 (4 words expressed as 8-number embedded vectors). A single output neuron is connected to the embedding layer by 33 weights (32 from the embedding layer and a single bias neuron). Because this is a single-class classification network, we use the sigmoid activation function and binary_crossentropy.

The program now trains the neural network. The embedding lookup and dense 33 weights are updated to produce a better score.


In [ ]:
# Program neural network

criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters())

# Training the model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(padded_reviews.long())
    loss = criterion(outputs.squeeze(), torch.tensor(labels, dtype=torch.float))
    loss.backward()
    optimizer.step()

We can see the learned embeddings. Think of each word's vector as a location in the 8 dimension space where words associated with positive reviews are close to other words. Similarly, training places negative reviews close to each other. In addition to the training setting these embeddings, the 33 weights between the embedding layer and output neuron similarly learn to transform these embeddings into an actual prediction. You can see these embeddings here.

In [ ]:
# Print out learned embeddings

embedding_weights = list(model[0].parameters())[0]
print(embedding_weights.shape)
print(embedding_weights)


We can now evaluate this neural network's accuracy, including the embeddings and the learned dense layer.


In [ ]:
# Evaluate neural network

with torch.no_grad():
    outputs = model(padded_reviews.long())
    predictions = (outputs > 0.5).float().squeeze()
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    loss_value = criterion(outputs.squeeze(), torch.tensor(labels, dtype=torch.float)).item()

print(f'Accuracy: {accuracy}')
print(f'Log-loss: {loss_value}')

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image54B.png)

The accuracy is great, but there could be overfitting. It would be good to use early stopping to not overfit for a more complex data set. However, the loss is not perfect. Even though the predicted probabilities indicated a correct prediction in every case, the program did not achieve absolute confidence in each correct answer. The lack of confidence was likely due to the small amount of noise (previously discussed) in the data set. Some words that appeared in both positive and negative reviews contributed to this lack of absolute certainty.


# **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Copy of Class_04_1.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas. If you want your lesson graded you must turn in a PDF of a COPY of this lesson that was saved on your GDrive, not the original Colab notebook.

## **Lizard Tail**

## **UNIVAC**

![___](https://upload.wikimedia.org/wikipedia/commons/2/2f/Univac_I_Census_dedication.jpg)

**UNIVAC (Universal Automatic Computer)** was a line of electronic digital stored-program computers starting with the products of the Eckert–Mauchly Computer Corporation. Later the name was applied to a division of the Remington Rand company and successor organizations.

The BINAC, built by the Eckert–Mauchly Computer Corporation, was the first general-purpose computer for commercial use, but it was not a success. The last UNIVAC-badged computer was produced in 1986.

**UNIVAC Sperry Rand label**

J. Presper Eckert and John Mauchly built the ENIAC (Electronic Numerical Integrator and Computer) at the University of Pennsylvania's Moore School of Electrical Engineering between 1943 and 1946. A 1946 patent rights dispute with the university led Eckert and Mauchly to depart the Moore School to form the Electronic Control Company, later renamed Eckert–Mauchly Computer Corporation (EMCC), based in Philadelphia, Pennsylvania. That company first built a computer called BINAC (BINary Automatic Computer) for Northrop Aviation (which was little used, or perhaps not at all). Afterwards, the development of UNIVAC began in April 1946.[1] UNIVAC was first intended for the Bureau of the Census, which paid for much of the development, and then was put in production.

With the death of EMCC's chairman and chief financial backer Henry L. Straus in a plane crash on October 25, 1949, EMCC was sold to typewriter, office machine, electric razor, and gun maker Remington Rand on February 15, 1950. Eckert and Mauchly now reported to Leslie Groves, the retired army general who had previously managed building The Pentagon and led the Manhattan Project.

The most famous UNIVAC product was the UNIVAC I mainframe computer of 1951, which became known for predicting the outcome of the U.S. presidential election the following year: this incident is noteworthy because the computer correctly predicted an Eisenhower landslide over Adlai Stevenson, whereas the final Gallup poll had Eisenhower winning the popular vote 51–49 in a close contest.

The prediction led CBS's news boss in New York, Siegfried Mickelson, to believe the computer was in error, and he refused to allow the prediction to be read. Instead, the crew showed some staged theatrics that suggested the computer was not responsive, and announced it was predicting 8–7 odds for an Eisenhower win (the actual prediction was 100–1 in his favour).

When the predictions proved true—Eisenhower defeated Stevenson in a landslide, with UNIVAC coming within 3.5% of his popular vote total and four votes of his Electoral College total—Charles Collingwood, the on-air announcer, announced that they had failed to believe the earlier prediction.

The United States Army requested a UNIVAC computer from Congress in 1951. Colonel Wade Heavey explained to the Senate subcommittee that the national mobilization planning involved multiple industries and agencies: "This is a tremendous calculating process...there are equations that can not be solved by hand or by electrically operated computing machines because they involve millions of relationships that would take a lifetime to figure out." Heavey told the subcommittee it was needed to help with mobilization and other issues similar to the invasion of Normandy that were based on the relationships of various groups.

The UNIVAC was manufactured at Remington Rand's former Eckert-Mauchly Division plant on W Allegheny Avenue in Philadelphia, Pennsylvania. Remington Rand also had an engineering research lab in Norwalk, Connecticut, and later bought Engineering Research Associates (ERA) in St. Paul, Minnesota. In 1953 or 1954 Remington Rand merged their Norwalk tabulating machine division, the ERA "scientific" computer division, and the UNIVAC "business" computer division into a single division under the UNIVAC name. This severely annoyed those who had been with ERA and with the Norwalk laboratory.

In 1955 Remington Rand merged with Sperry Corporation to become Sperry Rand. General Douglas MacArthur, then the chairman of the Board of Directors of Remington Rand, was chosen to continue in that role in the new company. Harry Franklin Vickers, then the President of Sperry Corporation, continued as president and CEO of Sperry Rand. The UNIVAC division of Remington Rand was renamed the Remington Rand Univac division of Sperry Rand. William Norris was put in charge as Vice-President and General Manager reporting to the President of the Remington Rand Division (of Sperry Rand).

### **UNIVAC: Historical Development and Significance**

**Introduction**

UNIVAC (Universal Automatic Computer) was the first commercially available computer in the United States, marking a pivotal moment in the history of computing. Developed in the early 1950s, UNIVAC played a crucial role in transitioning computing from experimental laboratories to practical business and government applications.

**Origins and Development**

### Eckert and Mauchly

UNIVAC was developed by **J. Presper Eckert** and **John Mauchly**, the same engineers who created the **ENIAC** (Electronic Numerical Integrator and Computer), the first general-purpose electronic digital computer. After ENIAC, they founded the **Eckert-Mauchly Computer Corporation** in 1946 with the goal of producing a more advanced and commercially viable computer.

### Design Goals

UNIVAC was designed to:
- Handle both numeric and alphabetic data.
- Be suitable for business and administrative use.
- Automate data processing tasks traditionally performed by punch card machines.

## Key Milestones

### UNIVAC I (1951)

- **First Delivered**: To the U.S. Census Bureau on **June 14, 1951**.
- **Technology**: Used vacuum tubes, mercury delay lines for memory, and magnetic tape for storage.
- **Speed**: Could perform approximately 1,000 calculations per second.
- **Input/Output**: Featured a typewriter-like console and tape drives.

### Commercial Impact
- **Remington Rand Acquisition**: In 1950, Eckert-Mauchly was acquired by Remington Rand, which marketed UNIVAC.
- **Presidential Election Prediction**: UNIVAC I famously predicted the outcome of the 1952 U.S. presidential election on live television, correctly forecasting Eisenhower's victory—demonstrating the power of computing to the public.

## Technical Specifications

| Feature              | Specification                          |
|----------------------|----------------------------------------|
| Memory               | 1,000 words (12 characters each)       |
| Word Size            | 72 bits                                |
| Clock Speed          | 2.25 MHz                               |
| Storage              | Magnetic tape                          |
| Programming Language | Machine code                           |

## Legacy and Influence

UNIVAC's success helped establish the viability of computers in business and government. It influenced the development of subsequent systems and contributed to the growth of the American computer industry.

### Successors

- **UNIVAC II**: Introduced in 1958 with improved memory and performance.
- **UNIVAC 1100 Series**: Became popular in the 1960s and 1970s for scientific and business applications.

### Cultural Impact

UNIVAC became a symbol of modernity and technological progress in the 1950s. Its televised election prediction helped demystify computers and sparked public interest in computing.

## Conclusion

UNIVAC was more than just a machine—it was a milestone in the evolution of computing. By bridging the gap between theoretical computing and practical application, it laid the foundation for the digital age.

## References

- Ceruzzi, Paul E. *A History of Modern Computing*. MIT Press.
- U.S. Census Bureau Archives
- Computer History Museum
